# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [101]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

<b>Store Part I Results into a DataFrame</b>
<ul><li>Load csv exported in Part I to a Dataframe</li></ul>


In [102]:
# Load and Store the Data from weatherpy into a dataframe

cities_path = "../Weatherpy/output_data/cities_output.csv"
cities_df = pd.read_csv(cities_path)
cities_df.head()

,city,country,city_id,date,latitude,longitude,cloudiness,description,temperature,feels_like,temp_min,temp_max,humidity,visbility,windspeed
0,Nemuro,JP,2128975,1648789984,43.3236,145.5750,100,overcast clouds,29.59,20.52,29.59,29.59,77,10000,10.42
1,Luganville,VU,2136150,1648789984,-15.5333,167.1667,100,heavy intensity rain,83.08,92.17,83.08,83.08,82,10000,2.89
2,Balkanabat,TM,161616,1648789984,39.5108,54.3671,0,clear sky,60.35,57.69,60.35,60.35,34,10000,2.68
3,Chokurdakh,RU,2126123,1648789763,70.6333,147.9167,17,few clouds,-7.08,-15.47,-7.08,-7.08,94,10000,3.24
4,Vaini,TO,4032243,1648789749,-21.2000,-175.2000,75,broken clouds,80.76,88.61,80.76,80.76,94,10000,4.61


<b>Humidity Heatmap</b>
<ul>
    <li> Configure GMAP </li>
    <li> Use the Lat and Lng as Locations and Humidity as the Weight </li>
    <li> Add Heatmap layer to map </li>
</ul>

In [122]:
gmaps.configure(api_key = g_key)

city_coord = cities_df[['latitude', 'longitude']].astype(float)
humidity = cities_df['humidity'].astype(float)

layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'}

fig = gmaps.figure(map_type = 'HYBRID', layout = layout)
heat = gmaps.heatmap_layer(city_coord, weights = humidity, dissipating = False, max_intensity = 300, point_radius = 5)


fig.add_layer(heat)
fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1000px'))

<b> Create new DataFrame fitting weather criteria </b>
<ul>
    <li> Narrow down the cities to fit weather conditions </li>
    <li> Drop any rows with null values </li>
</ul>

In [112]:
# create copy of city data to manipulate in pandas
copy_city_data = cities_df.copy()

# select the data that has a Max Temperature of 80 degrees but higher than 70
temp_optimized_df = copy_city_data[(copy_city_data['temp_max'] > 70) & (copy_city_data['temp_max'] < 80)]

# select data with wind speed less than 10 mph
wind_optimized_df =temp_optimized_df[(temp_optimized_df['windspeed'] < 10)]


# select data with zero cloudiness
cloud_optimized_df = wind_optimized_df[(wind_optimized_df['cloudiness'] ==0)]

# drop any data data that doesn't satisfy all three conditions
weather_optimized_df = cloud_optimized_df.dropna(how='any')
print(len(weather_optimized_df))

9


<b> Hotel Map </b>
<ul>
    <li>Store into variable named hotel_df</li>
    <li>Add a "Hotel Name" column to the DataFrame</li>
    <li>Set parameters to search for hotels w/ 5000 meters</li>
    <li>Hit the Google Places API for each city's coodinates</li>
    <li>Store the first Hotel result into the DataFrame</li>
    <li>Plot markers on top of the heatmap</li>
</ul>

In [113]:
# use Google Places API to find first hotel within 5000 meters of coords
hotel_df = cloud_optimized_df[['city', 'country', 'latitude', 'longitude']]



In [114]:
# params dictionary to update each iteration
params = {
    'radius': 5000,
    'types': 'hotel',
    'keyword': 'hotel',
    'key': g_key
}

# use the lat/long we recovered to get the hotels
for index, row in hotel_df.iterrows():
    lat = row['latitude']
    lng = row['longitude']
    params['location'] = f"{lat}, {lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")




Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [115]:
info_box_template = """
    <dl>
    <dt>Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{city}</dd>
    <dt>Country</dt><dd>{country}</dd>
    </dl>"""


hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = ten_hotel_df[['latitude', 'longitude']]


In [123]:
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1000px'))